# Contributors
Zijian Zhang 14851598 $\\$
Amish Sewlal 13297503 $\\$
Ravi Sharma Kaushik 14332744

# Imports

In [84]:
import numpy as np
import matplotlib.pyplot as plt
from Mandelbrot import *
from MonteCarloIntegration import *

# Creating Mandlebrot Set

# Area of Mandlebrot Set using Monte Carlo Method

# Convergence study for various number of iterations

In [86]:
# Parameters
low = -2
high = 2
samples = 100**2
N = 50  # Number of simulations
iterations = np.arange(10, 500, 5)

# Simulating
results = []
for i in iterations:
    area = []
    for _ in range(N):
        A = randomSampling(low, high, i, samples)
        area.append(A)
    
    # Compute mean and standard deviation
    A_mean = np.mean(area)
    A_std = np.std(area, ddof=1)
    
    # Add to results
    results.append([A_mean, A_std])

# Convert to numpy array
results = np.array(results)

10
15
20
25
30
35


KeyboardInterrupt: 

In [ ]:
# Get mean and 95% confidence interval
A = results[:, 0]
A_err = results[:, 1]

# Get difference
dA = abs(A[-1] - A)
dA_err = (A_err[-1]**2  + A_err**2)**0.5 * 1.96 / np.sqrt(N)

# Left and right limit for confidence interval
left = dA - dA_err
right = dA + dA_err
left[left < 0] = 0

# Plotting
plt.plot(iterations, dA, color='red')

# Adding shaded area to show standard deviation
plt.fill_between(iterations, dA - dA_err, dA + dA_err, color='red', alpha=0.3)

# Setting for plot
plt.grid()
plt.xlabel('$Iterations$')
plt.ylabel('$|A_{j,s} - A_{i,s}|$')

# Convergence study for various number of samples

In [ ]:
# Parameters
low = -2
high = 2
iterations = 200
samples = np.arange(5, 100, 2) ** 2
N = 50  # Number of simulations

# Simulating
results = []
for sample in samples:
    area = []
    for _ in range(N):
        A = randomSampling(low, high, iterations, sample)
        area.append(A)

    # Compute mean and standard deviation
    A_mean = np.mean(area)
    A_std = np.std(area, ddof=1)

    # Add to results
    results.append([A_mean, A_std])

# Convert to numpy array
results = np.array(results)

In [ ]:
# Get mean and 95% confidence interval
A = results[:, 0]
A_err = results[:, 1]

# Get difference
dA = abs(A[-1] - A)
dA_err = (A_err[-1]**2  + A_err**2)**0.5 * 1.96 / np.sqrt(N)

# Left and right limit for confidence interval
left = dA - dA_err
right = dA + dA_err
left[left < 0] = 0

# Plotting
plt.plot(samples, dA, color='red')

# Adding shaded area to show standard deviation
plt.fill_between(samples, left, right, color='red', alpha=0.3)

# Setting for plot
plt.grid()
plt.xlabel('$Samples$')
plt.ylabel('$|A_{j,s} - A_{i,s}|$')

# Using various sampling techniques  

# Approach for improving convergence